### Read CSV

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
csv_headers = ["center", "left", "right", "steering", "throttle", "brake", "speed"]

data = pd.read_csv('CarSim_data/driving_log.csv', names=csv_headers)

In [ ]:
data.head()

### Read Image, Crop and Resize

In [ ]:
for i in data['center']:
    print(i)

### Features

In [ ]:
img_path = data['center'][1]
#print(img_path)
num_samples = len(data)
i=0
for row in data['center']:
    #print(i)
    I = cv2.imread(img_path)
    b,g,r = cv2.split(I)
    I = cv2.merge([r,g,b])
    #cv2.imwrite('I.jpg',I)
    I = I[50:,:]
    I = cv2.resize(I,None,fx=0.25,fy=0.25,interpolation=cv2.INTER_AREA)
    plt.imshow(I)
    
    # https://carnd-forums.udacity.com/questions/32716893/
    # does-anyone-know-how-to-make-multiple-images-into-4d-arrays-like-pickled-data#
    #arr = np.zeros((num_examples, width, height, channels))

    X_train = np.zeros((num_samples, I.shape[0], I.shape[1], I.shape[2]))
    X_train[i, :, :, :] = I
    i=i+1

### Labels

In [ ]:
# Convert a pandas dataframe (df) to a numpy ndarray
y_train = data['steering'].values

### Create Features and Labels

### Preprocess the Data
1.) Shuffle the data

2.) Normalize the features using Min-Max scaling between -0.5 and 0.5

3.) Steering Angle: -25 to 25


In [ ]:
data.shape
len(data)

In [ ]:
I.shape[2]

In [ ]:
# https://carnd-forums.udacity.com/questions/32716893/
# does-anyone-know-how-to-make-multiple-images-into-4d-arrays-like-pickled-data#
#arr = np.zeros((num_examples, width, height, channels))

features = np.zeros((len(data), I.shape[1], I.shape[0], I.shape[2]))
features[i, :, :, :] = I


In [ ]:
arr.shape